In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import phoenix as px
from phoenix.evals import OpenAIModel, llm_classify, TOOL_CALLING_PROMPT_TEMPLATE
from phoenix.experiments import run_experiment, evaluate_experiment
from phoenix.experiments.types import Example
from phoenix.experiments.evaluators import create_evaluator
from phoenix.otel import register
import pandas as pd
from helper import get_openai_api_key, process_messages
from init_phoenix import init_phoenix
from tutorial1_code import Agent
from prompts.eval import ENTITY_CORRECTNESS_LLM_JUDGE_PROMPT, CLARITY_LLM_JUDGE_PROMPT

# from helper import process_messages, update_sql_gen_prompt

from datetime import datetime
import json

import os
import nest_asyncio
nest_asyncio.apply()

In [3]:
open_ai_key = get_openai_api_key()
eval_model=OpenAIModel(model="gpt-4o", api_key=open_ai_key)

In [4]:
px_client = px.Client()

In [5]:
PROJECT_NAME = "overall_experiment"

In [6]:
client, tool_calling_client, tracer = init_phoenix(project_name=PROJECT_NAME)

OpenTelemetry Tracing Details
|  Phoenix Project: overall_experiment
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
agent = Agent(client, tool_calling_client, tracer)

In [ ]:
overall_experiment_questions = [
    {'question': 'What was the most popular product SKU?',
     'sql_result': '   SKU_Coded  Total_Qty_Sold 0    6200700         52262.0', 
     'sql_generated': '```sql\nSELECT SKU_Coded, SUM(Qty_Sold) AS Total_Qty_Sold\nFROM sales\nGROUP BY SKU_Coded\nORDER BY Total_Qty_Sold DESC\nLIMIT 1;\n```'},
    {'question': 'What was the total revenue across all stores?', 
     'sql_result': '   Total_Revenue 0   1.327264e+07', 
     'sql_generated': '```sql\nSELECT SUM(Total_Sale_Value) AS Total_Revenue\nFROM sales;\n```'},
    {'question': 'Which store had the highest sales volume?',
     'sql_result': '   Store_Number  Total_Sales_Volume 0          2970             59322.0', 
     'sql_generated': '```sql\nSELECT Store_Number, SUM(Total_Sale_Value) AS Total_Sales_Volume\nFROM sales\nGROUP BY Store_Number\nORDER BY Total_Sales_Volume DESC\nLIMIT 1;\n```'},
    {'question': 'Create a bar chart showing total sales by store',
     'sql_result': '    Store_Number    Total_Sales 0            880  420302.088397 1           1650  580443.007953 2           4180  272208.118542 3            550  229727.498752 4           1100  497509.528013 5           3300  619660.167018 6           3190  335035.018792 7           2970  836341.327191 8           3740  359729.808228 9           2530  324046.518720 10          4400   95745.620250 11          1210  508393.767785 12           330  370503.687331 13          2750  453664.808068 14          1980  242290.828499 15          1760  350747.617798 16          3410  410567.848126 17           990  378433.018639 18          4730  239711.708869 19          4070  322307.968330 20          3080  495458.238811 21          2090  309996.247965 22          1320  592832.067579 23          2640  308990.318559 24          1540  427777.427815 25          4840  389056.668316 26          2860  132320.519487 27          2420  406715.767402 28           770  292968.918642 29          3520  145701.079372 30           660  343594.978075 31          3630  405034.547846 32          2310  412579.388504 33          2200  361173.288199 34          1870  401070.997685', 
     'sql_generated': '```sql\nSELECT Store_Number, SUM(Total_Sale_Value) AS Total_Sales\nFROM sales\nGROUP BY Store_Number;\n```'},
    {'question': 'What percentage of items were sold on promotion?',
     'sql_result': '   Promotion_Percentage 0              0.625596',
     'sql_generated': "```sql\nSELECT \n    (SUM(CASE WHEN On_Promo = 'Yes' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS Promotion_Percentage\nFROM \n    sales;\n```"},
    {'question': 'What was the average transaction value?',
     'sql_result': '   Average_Transaction_Value 0                  19.018132',
     'sql_generated': '```sql\nSELECT AVG(Total_Sale_Value) AS Average_Transaction_Value\nFROM sales;\n```'},
    {'question': 'Create a line chart showing sales in 2021',
     'sql_result': '  sale_month  total_quantity_sold  total_sales_value 0 2021-11-01              43056.0      499984.428193 1 2021-12-01              75724.0      910982.118423', 
     'sql_generated': '```sql\nSELECT MONTH(Sold_Date) AS Month, SUM(Total_Sale_Value) AS Total_Sales\nFROM sales\nWHERE YEAR(Sold_Date) = 2021\nGROUP BY MONTH(Sold_Date)\nORDER BY MONTH(Sold_Date);\n```'}
]

overall_experiment_df = pd.DataFrame(overall_experiment_questions)

now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

dataset = px_client.get_dataset(dataset_name="overall_experiment_inputs-2025-07-08 16:16:15")
# create a dataset consisting of input questions and expected outputs
# dataset = px_client.upload_dataset(dataframe=overall_experiment_df, 
#                                    dataset_name=f"overall_experiment_inputs-{now}", 
#                                    input_keys=["question"], 
#                                    output_keys=["sql_result", "sql_generated"])

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDoxMA==/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246MTA=


In [9]:
# evaluator for the router
def function_calling_eval(input: str, output: str) -> float:
    if output is None:
        return 0
    function_calls = output.get("tool_calls")
    if function_calls:
        eval_df = pd.DataFrame({
            "question": [input.get("question")] * len(function_calls),
            "tool_call": function_calls
        })
            
        tool_call_eval = llm_classify(
            data = eval_df,
            template = TOOL_CALLING_PROMPT_TEMPLATE.template[0].template.replace("{tool_definitions}", 
                                                                                 json.dumps(agent.tools).replace("{", '"').replace("}", '"')),
            rails = ['correct', 'incorrect'],
            model=eval_model,
            provide_explanation=True
        )

        tool_call_eval['score'] = tool_call_eval.apply(lambda x: 1 if x['label']=='correct' else 0, axis=1)
        return tool_call_eval['score'].mean()
    else:
        return 0

In [10]:
# evaluator for tool 1: database lookup
def evaluate_sql_result(output, expected) -> bool:    
    if output is None:
        return False
    sql_result = output.get("tool_responses")
    if not sql_result:
        return True
    
    # Find first lookup_sales_data response
    sql_result = next((r for r in sql_result if r.get("tool_name") == "lookup_sales_data"), None)
    if not sql_result:
        return True
        
    # Get the first response
    sql_result = sql_result.get("tool_response", "")

    # Extract just the numbers from both strings
    result_nums = ''.join(filter(str.isdigit, sql_result))
    expected_nums = ''.join(filter(str.isdigit, expected.get("sql_result")))
    return result_nums == expected_nums

In [11]:
# evaluator for tool 2: data analysis
def evaluate_clarity(output: str, input: str) -> bool:
    if output is None:
        return False
    df = pd.DataFrame({"query": [input.get("question")],
                       "response": [output.get("final_output")]})
    response = llm_classify(
        data=df,
        template=CLARITY_LLM_JUDGE_PROMPT,
        rails=["clear", "unclear"],
        model=eval_model,
        provide_explanation=True
    )
    return response['label'] == 'clear'

In [12]:
# evaluator for tool 2: data analysis
def evaluate_entity_correctness(output: str, input: str) -> bool:
    if output is None:
        return False
    df = pd.DataFrame({"query": [input.get("question")], 
                       "response": [output.get("final_output")]})
    response = llm_classify(
        data=df,
        template=ENTITY_CORRECTNESS_LLM_JUDGE_PROMPT,
        rails=["correct", "incorrect"],
        model=eval_model,
        provide_explanation=True
    )
    return response['label'] == 'correct'

In [13]:
# evaluator for tool 3: data visualization   
def code_is_runnable(output: str) -> bool:
    """Check if the code is runnable"""
    if output is None:
        return False
    generated_code = output.get("tool_responses")
    if not generated_code:
        return True
    
    # Find first lookup_sales_data response
    generated_code = next((r for r in generated_code if r.get("tool_name") == "generate_visualization"), None)
    if not generated_code:
        return True
        
    # Get the first response
    generated_code = generated_code.get("tool_response", "")
    generated_code = generated_code.strip()
    generated_code = generated_code.replace("```python", "").replace("```", "")
    try:
        exec(generated_code)
        return True
    except Exception as e:
        return False

In [14]:
def run_agent_task(example: Example) -> str:
    print("Starting agent with messages:", example.input.get("question"))
    messages = [{"role": "user", "content": example.input.get("question")}]
    ret, messages = agent.run_agent(messages)
    return process_messages(messages)

In [ ]:
experiment = run_experiment(dataset,
                            run_agent_task,
                            evaluators=[function_calling_eval,
                                        evaluate_sql_result, 
                                        evaluate_clarity, 
                                        evaluate_entity_correctness, 
                                        code_is_runnable],
                            experiment_name="Overall Experiment v3",
                            experiment_description="Evaluating the overall experiment")

🧪 Experiment started.
📺 View dataset experiments: http://localhost:6006/datasets/RGF0YXNldDoxMA==/experiments
🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDoxMA==/compare?experimentId=RXhwZXJpbWVudDoxMw==


running tasks |          | 0/7 (0.0%) | ⏳ 00:00<? | ?it/s

Starting agent with messages: What was the most popular product SKU?
Running agent with messages: [{'role': 'user', 'content': 'What was the most popular product SKU?'}]
Starting router call span
Making router call to OpenAI
[{'role': 'user', 'content': 'What was the most popular product SKU?'}, {'role': 'system', 'content': '\nYou are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset.\n'}]
{'content': None, 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_J1mqIaZoQyIs77BAd2fBItsv', 'function': {'arguments': '{"prompt":"What was the most popular product SKU?"}', 'name': 'lookup_sales_data'}, 'type': 'function'}]}
Received response with tool calls: True
Starting tool calls span
lookup_sales_data
Starting router call span
Making router call to OpenAI
[{'role': 'user', 'content': 'What was the most popular product SKU?'}, {'role': 'system', 'content': '\nYou are 

running tasks |█▍        | 1/7 (14.3%) | ⏳ 00:23<02:18 | 23.08s/it

Starting agent with messages: Create a bar chart showing total sales by store
Running agent with messages: [{'role': 'user', 'content': 'Create a bar chart showing total sales by store'}]
Starting router call span
Making router call to OpenAI
[{'role': 'user', 'content': 'Create a bar chart showing total sales by store'}, {'role': 'system', 'content': '\nYou are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset.\n'}]
{'content': None, 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_WXVbWApjSE0xH1h6V17zxSNb', 'function': {'arguments': '{"prompt":"Create a bar chart showing total sales by store."}', 'name': 'lookup_sales_data'}, 'type': 'function'}]}
Received response with tool calls: True
Starting tool calls span
lookup_sales_data
Starting router call span
Making router call to OpenAI
[{'role': 'user', 'content': 'Create a bar chart showing total sales by stor

...

{'content': "It seems there is a persistent issue accessing the sales data for the year 2021. I'll try to obtain the data again. Please hold on.", 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_VcxRWRAS93eOzC6LjyQQZtRB', 'function': {'arguments': '{}', 'name': 'lookup_sales_data'}, 'type': 'function'}]}
Received response with tool calls: True
Starting tool calls span
Traceback (most recent call last):
  File "c:\Users\Valentin\Desktop\nextvision\eval_learning\.venv\Lib\site-packages\phoenix\experiments\functions.py", line 409, in async_run_experiment
    _output = task(*bound_task_args.args, **bound_task_args.kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Valentin\AppData\Local\Temp\ipykernel_21216\608730828.py", line 4, in run_agent_task
    ret, messages = agent.run_agent(messages)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Valentin\Desktop\nex

running tasks |██████████| 7/7 (100.0%) | ⏳ 01:57<00:00 | 16.77s/it


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |█▏        | 4/35 (11.4%) | ⏳ 00:03<00:26 |  1.17it/s
running experiment evaluations |█▋        | 6/35 (17.1%) | ⏳ 00:08<00:44 |  1.53s/it



llm_classify |██████████| 4/4 (100.0%) | ⏳ 00:11<00:00 |  2.86s/it
llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:04<00:00 |  4.63s/it
running experiment evaluations |██▎       | 8/35 (22.9%) | ⏳ 00:15<00:58 |  2.17s/it
llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:10<00:00 | 10.62s/it
llm_classify |██████████| 5/5 (100.0%) | ⏳ 00:03<00:00 |  1.39it/s
running experiment evaluations |██▊       | 10/35 (28.6%) | ⏳ 00:21<00:58 |  2.33s/it
running experiment evaluations |███▍      | 12/35 (34.3%) | ⏳ 00:27<01:00 |  2.63s/it









llm_classify |██████████| 7/7 (100.0%) | ⏳ 00:03<00:00 |  2.32it/s
running experiment evaluations |████▎     | 15/35 (42.9%) | ⏳ 00:33<00:43 |  2.15s/it


llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:10<00:00 | 10.20s/it
llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:10<00:00 | 10.19s/it


ll


🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDoxMA==/compare?experimentId=RXhwZXJpbWVudDoxMw==

Experiment Summary (07/08/25 04:19 PM +0200)
--------------------------------------------
                     evaluator  n  n_scores  avg_score  n_labels  \
0             code_is_runnable  7         7   0.714286         7   
1             evaluate_clarity  7         7   0.142857         2   
2  evaluate_entity_correctness  7         7   0.571429         2   
3          evaluate_sql_result  7         7   0.000000         7   
4        function_calling_eval  7         7   0.000000         0   

              top_2_labels  
0  {'True': 5, 'False': 2}  
1             {'False': 2}  
2             {'False': 2}  
3             {'False': 7}  
4                     None  

Tasks Summary (07/08/25 04:18 PM +0200)
---------------------------------------
   n_examples  n_runs  n_errors  \
0           7       7         2   

                                           top_error  
0  T